<a href="https://colab.research.google.com/github/DjordjeRadovanovic/PPPO_Projekat_IT39_2020/blob/main/PPPO_Projekat_IT39_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Principi prezentacije i prepoznavanja oblika 2023/2024**


**Đorđe Radovanović IT39/2020**

**Tema:** Klasifikacija lišća

## **Sadržaj**

1. Uvod
2. Tema
3. Učitavanje dataseta
4. Projekat
5. Zaključak
6. Literatura

## **1. Uvod**

## **2. Tema**

## **3. Učitavanje dataseta**

## **4. Projekat**

## **5. Zaključak**

## **6. Literatura**